In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
import tensorflow as tf

In [9]:
np.random.seed(3141593)
tf.set_random_seed(3141593)
data = pd.read_csv("../input/leaf-classification/train.csv")
data.species = data.species.astype('str').astype('category')
output_nodes = data.species.nunique()
data.id = data.id.apply(lambda x: "../input/preprocessed-images/preprocessed_images/preprocessed_images/"+str(x)+".jpg")
trainset, testset = train_test_split(data, test_size=0.1, stratify=data.species, random_state=3141593)
del data

In [5]:
onehot = LabelBinarizer()
y_train = onehot.fit_transform(trainset.species)
y_test = onehot.transform(testset.species)

In [14]:
img_size = 224
b_size = 32

train_gen = ImageDataGenerator(rescale=1/.255, width_shift_range=0.1, height_shift_range=0.1, rotation_range=0.1,
                               shear_range=0.1, zoom_range=0.1, horizontal_flip=True, vertical_flip=True)
test_gen=ImageDataGenerator(rescale=1/.255)

## mobilenet only for RGB - ImageDataGenerator converts grayscale to rgb
train_generator = train_gen.flow_from_dataframe(trainset, directory=None, x_col='id', y_col='species', 
                                                target_size=(img_size,img_size), color_mode='rgb',
                                                class_mode='categorical', batch_size=b_size)

test_generator = test_gen.flow_from_dataframe(testset, directory=None, x_col='id', y_col='species', 
                                                target_size=(img_size,img_size), color_mode='rgb',
                                                class_mode='categorical', batch_size=b_size)

Found 891 images belonging to 99 classes.
Found 99 images belonging to 99 classes.


In [15]:
weights = "../input/mobilenet/mobilenet_1_0_224_tf_no_top.h5"

base_model = MobileNet(input_shape=((img_size,img_size,3)), dropout=0.2, include_top=False, weights=weights, 
                      pooling='max')
output_model = Dense(output_nodes, activation='softmax')(base_model.output)
model = Model(inputs=base_model.input, outputs=output_model)

model.compile(Adam(lr=1e-4, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit_generator(train_generator, steps_per_epoch=10, epochs=400,
                    validation_data=test_generator, validation_steps=3)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/400
10/10 [==============================] - 9s 888ms/step - loss: 12.9708 - acc: 0.0250 - val_loss: 11.1934 - val_acc: 0.0312
Epoch 2/400
10/10 [==============================] - 2s 238ms/step - loss: 10.6608 - acc: 0.0375 - val_loss: 9.8531 - val_acc: 0.0746
Epoch 3/400
10/10 [==============================] - 4s 414ms/step - loss: 8.9574 - acc: 0.0861 - val_loss: 8.0049 - val_acc: 0.1194
Epoch 4/400
10/10 [==============================] - 4s 416ms/step - loss: 6.6941 - acc: 0.1344 - val_loss: 6.7900 - val_acc: 0.1343
Epoch 5/400
10/10 [==============================] - 4s 415ms/step - loss: 6.4806 - acc: 0.1875 - val_loss: 6.0156 - val_acc: 0.1354
Epoch 6/400
10/10 [==============================] - 4s 404ms/step - loss: 5.5853 - acc: 0.2154 - val_loss: 5.0186 - val_acc: 0.2388
Epoch 7/400
10/10 [==============================] - 4s 408ms/step - loss: 4.8815 - acc: 0.2687 - val_loss: 5.8468 - val_

10/10 [==============================] - 4s 413ms/step - loss: 2.4797 - acc: 0.8065 - val_loss: 2.4362 - val_acc: 0.8021
Epoch 122/400
10/10 [==============================] - 4s 406ms/step - loss: 1.9363 - acc: 0.8250 - val_loss: 2.7016 - val_acc: 0.7910
Epoch 123/400
10/10 [==============================] - 4s 417ms/step - loss: 3.0280 - acc: 0.7438 - val_loss: 2.9797 - val_acc: 0.7761
Epoch 124/400
10/10 [==============================] - 4s 403ms/step - loss: 2.1090 - acc: 0.8080 - val_loss: 2.1095 - val_acc: 0.7910
Epoch 125/400
10/10 [==============================] - 4s 439ms/step - loss: 2.3975 - acc: 0.7906 - val_loss: 2.7693 - val_acc: 0.7604
Epoch 126/400
10/10 [==============================] - 5s 456ms/step - loss: 2.9654 - acc: 0.7653 - val_loss: 1.8864 - val_acc: 0.8060
Epoch 127/400
10/10 [==============================] - 5s 454ms/step - loss: 2.1757 - acc: 0.8313 - val_loss: 3.2873 - val_acc: 0.6866
Epoch 128/400
10/10 [==============================] - 4s 411ms/step 

Epoch 182/400
10/10 [==============================] - 4s 399ms/step - loss: 2.1790 - acc: 0.8294 - val_loss: 2.9836 - val_acc: 0.7164
Epoch 183/400
10/10 [==============================] - 5s 473ms/step - loss: 2.3204 - acc: 0.8219 - val_loss: 1.7068 - val_acc: 0.8209
Epoch 184/400
10/10 [==============================] - 5s 468ms/step - loss: 2.4598 - acc: 0.8031 - val_loss: 3.1650 - val_acc: 0.7164
Epoch 185/400
10/10 [==============================] - 4s 418ms/step - loss: 2.5156 - acc: 0.8226 - val_loss: 2.6572 - val_acc: 0.7604
Epoch 186/400
10/10 [==============================] - 4s 409ms/step - loss: 2.1299 - acc: 0.8344 - val_loss: 3.2593 - val_acc: 0.7015
Epoch 187/400
10/10 [==============================] - 4s 408ms/step - loss: 2.4770 - acc: 0.8000 - val_loss: 1.7755 - val_acc: 0.7612
Epoch 188/400
10/10 [==============================] - 4s 404ms/step - loss: 2.7362 - acc: 0.7940 - val_loss: 2.7144 - val_acc: 0.7463
Epoch 189/400
10/10 [==============================] - 

Epoch 243/400
10/10 [==============================] - 4s 413ms/step - loss: 2.2403 - acc: 0.8281 - val_loss: 2.2961 - val_acc: 0.7910
Epoch 244/400
10/10 [==============================] - 5s 485ms/step - loss: 1.7951 - acc: 0.8738 - val_loss: 2.8942 - val_acc: 0.7612
Epoch 245/400
10/10 [==============================] - 5s 484ms/step - loss: 2.4219 - acc: 0.8344 - val_loss: 2.6163 - val_acc: 0.7812
Epoch 246/400
10/10 [==============================] - 4s 414ms/step - loss: 2.3879 - acc: 0.8406 - val_loss: 2.0093 - val_acc: 0.8209
Epoch 247/400
10/10 [==============================] - 4s 404ms/step - loss: 2.6789 - acc: 0.8221 - val_loss: 2.9826 - val_acc: 0.7463
Epoch 248/400
10/10 [==============================] - 4s 405ms/step - loss: 1.6727 - acc: 0.8844 - val_loss: 2.4326 - val_acc: 0.7761
Epoch 249/400
10/10 [==============================] - 4s 417ms/step - loss: 1.9474 - acc: 0.8500 - val_loss: 2.5500 - val_acc: 0.7708
Epoch 250/400
10/10 [==============================] - 

Epoch 304/400
10/10 [==============================] - 4s 412ms/step - loss: 2.0866 - acc: 0.8406 - val_loss: 1.9202 - val_acc: 0.7910
Epoch 305/400
10/10 [==============================] - 4s 410ms/step - loss: 2.1090 - acc: 0.8375 - val_loss: 2.5023 - val_acc: 0.7812
Epoch 306/400
10/10 [==============================] - 4s 402ms/step - loss: 2.4797 - acc: 0.8252 - val_loss: 2.3279 - val_acc: 0.7910
Epoch 307/400
10/10 [==============================] - 4s 410ms/step - loss: 1.9515 - acc: 0.8656 - val_loss: 2.0208 - val_acc: 0.8209
Epoch 308/400
10/10 [==============================] - 4s 404ms/step - loss: 2.3135 - acc: 0.8258 - val_loss: 2.6135 - val_acc: 0.8209
Epoch 309/400
10/10 [==============================] - 4s 424ms/step - loss: 1.7926 - acc: 0.8781 - val_loss: 2.5290 - val_acc: 0.8021
Epoch 310/400
10/10 [==============================] - 4s 403ms/step - loss: 2.5647 - acc: 0.8250 - val_loss: 1.9164 - val_acc: 0.8358
Epoch 311/400
10/10 [==============================] - 

Epoch 365/400
10/10 [==============================] - 4s 418ms/step - loss: 1.6870 - acc: 0.8906 - val_loss: 2.4900 - val_acc: 0.8021
Epoch 366/400
10/10 [==============================] - 4s 412ms/step - loss: 2.8097 - acc: 0.8187 - val_loss: 2.0172 - val_acc: 0.8358
Epoch 367/400
10/10 [==============================] - 4s 401ms/step - loss: 2.1560 - acc: 0.8477 - val_loss: 1.9864 - val_acc: 0.8060
Epoch 368/400
10/10 [==============================] - 4s 409ms/step - loss: 2.1008 - acc: 0.8562 - val_loss: 3.0212 - val_acc: 0.7313
Epoch 369/400
10/10 [==============================] - 4s 412ms/step - loss: 2.2280 - acc: 0.8375 - val_loss: 2.4782 - val_acc: 0.7812
Epoch 370/400
10/10 [==============================] - 4s 409ms/step - loss: 2.1020 - acc: 0.8612 - val_loss: 2.3598 - val_acc: 0.7761
Epoch 371/400
10/10 [==============================] - 4s 411ms/step - loss: 2.3106 - acc: 0.8469 - val_loss: 2.3573 - val_acc: 0.8060
Epoch 372/400
10/10 [==============================] - 

In [17]:
test_generator = test_gen.flow_from_dataframe(testset, directory=None, x_col='id', y_col='species', 
                                                target_size=(img_size,img_size), color_mode='rgb',
                                                class_mode='categorical', batch_size=1)

loss, acc = model.evaluate_generator(test_generator, steps=len(test_generator))
print('loss:',loss)
print('acc:',acc)

Found 99 images belonging to 99 classes.
loss: 2.6669861996826696
acc: 0.7474747474747475
